<a href="https://colab.research.google.com/github/rickiepark/the-lm-book/blob/main/bow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="display: flex; justify-content: center;">
    <div style="background-color: #f4f6f7; padding: 15px; width: 80%;">
        <table style="width: 100%">
            <tr>
                <td style="vertical-align: middle;">
                    <span style="font-size: 14px;">
                        <a href="https://tensorflow.blog/the-lm-book" target="_blank" rel="noopener"><대규모 언어 모델, 핵심만 빠르게!>(인사이트, 2025)</a>의 주피터 노트북<br><br>
                        코드 저장소: <a href="https://github.com/rickiepark/the-lm-book" target="_blank" rel="noopener">https://github.com/rickiepark/the-lm-book</a>
                    </span>
                </td>
                <td style="vertical-align: middle;">
                    <a href="https://www.thelmbook.com" target="_blank" rel="noopener">
                        <img src="https://tensorflow.blog/wp-content/uploads/2025/10/cover-the-lm-book.jpg" width="80px" alt="대규모 언어 모델, 핵심만 빠르게!" border="1">
                    </a>
                </td>
            </tr>
        </table>
    </div>
</div>

In [1]:
import re, torch
import torch.nn as nn

torch.manual_seed(42)

In [2]:
docs = [
    "Movies are fun for everyone.",
    "Watching movies is great fun.",
    "Enjoy a great movie today.",
    "Research is interesting and important.",
    "Learning math is very important.",
    "Science discovery is interesting.",
    "Rock is great to listen to.",
    "Listen to music for fun.",
    "Music is fun for everyone.",
    "Listen to folk music!"
]

labels = [1, 1, 1, 3, 3, 3, 2, 2, 2, 2]
num_classes = len(set(labels))

num_classes

3

In [3]:
def tokenize(text):
    return re.findall(r"\w+", text.lower())

In [4]:
def get_vocabulary(texts):
    tokens = {token for text in texts for token in tokenize(text)}
    return {word: idx for idx, word in enumerate(sorted(tokens))}

In [5]:
vocabulary = get_vocabulary(docs)
vocabulary

{'a': 0,
 'and': 1,
 'are': 2,
 'discovery': 3,
 'enjoy': 4,
 'everyone': 5,
 'folk': 6,
 'for': 7,
 'fun': 8,
 'great': 9,
 'important': 10,
 'interesting': 11,
 'is': 12,
 'learning': 13,
 'listen': 14,
 'math': 15,
 'movie': 16,
 'movies': 17,
 'music': 18,
 'research': 19,
 'rock': 20,
 'science': 21,
 'to': 22,
 'today': 23,
 'very': 24,
 'watching': 25}

In [6]:
def doc_to_bow(doc, vocabulary):
    tokens = set(tokenize(doc))
    bow = [0] * len(vocabulary)
    for token in tokens:
        if token in vocabulary:
            bow[vocabulary[token]] = 1
    return bow

In [7]:
vectors = torch.tensor(
    [doc_to_bow(doc, vocabulary) for doc in docs],
    dtype=torch.float32
)
labels = torch.tensor(labels, dtype=torch.long) - 1
labels

tensor([0, 0, 0, 2, 2, 2, 1, 1, 1, 1])

In [8]:
input_dim = len(vocabulary)
hidden_dim = 50
output_dim = num_classes

In [9]:
class SimpleClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = SimpleClassifier(input_dim, hidden_dim, output_dim)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

for step in range(3000):
    optimizer.zero_grad()
    loss = criterion(model(vectors), labels)
    loss.backward()
    optimizer.step()

In [11]:
new_docs = [
    "Listening to rock music is fun.",
    "I love science very much."
]
class_names = ["Cinema", "Music", "Science"]

In [12]:
new_doc_vectors = torch.tensor(
    [doc_to_bow(new_doc, vocabulary) for new_doc in new_docs],
    dtype=torch.float32
)

with torch.no_grad():
    outputs = model(new_doc_vectors)
    predicted_ids = torch.argmax(outputs, dim=1) + 1

for i, new_doc in enumerate(new_docs):
    print(f'{new_doc}: {class_names[predicted_ids[i].item() - 1]}')

Listening to rock music is fun.: Music
I love science very much.: Science
